# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| export
import numpy as np
import networkx as nx
import math
import editdistance


def max_flow(G, current_node, destination):
    if current_node != destination:
        ss_weight, _ = nx.maximum_flow(G, current_node, destination)
    else:
        ss_weight = 0
    return ss_weight

def min_cost_max_flow(G, current_node, destination):
    if current_node != destination:
        mincostFlow = nx.max_flow_min_cost(G, current_node, destination)
        ss_weight = nx.cost_of_flow(G, mincostFlow)
    else:
        ss_weight = 0
    return ss_weight 

def levenshtein(G, current_node, destination):
    if current_node != destination:
        curr = list(G.neighbors(current_node))
        dest = list(G.neighbors(destination))
        ss_weight = editdistance.eval(curr, dest)
    else:
        ss_weight = 0
    return ss_weight

def jaccard_coefficient(G, current_node, destination):
    if current_node != destination:
        curr = list(G.neighbors(current_node))
        dest = list(G.neighbors(destination))
        denominator = len(curr)+len(dest)
        if denominator == 0:
            ss_weight = 0
        else:
            numerator = len(set(curr+dest))
            ss_weight = numerator/denominator
    else:
        ss_weight = 0
    return ss_weight 

def common_neighbors(G, current_node, destination):
    if current_node != destination:
        curr = list(G.neighbors(current_node))
        dest = list(G.neighbors(destination))
        ss_weight = len(set(curr+dest))
    else:
        ss_weight = 0
    return ss_weight 

def lhn_index(G, current_node, destination):
    if current_node != destination:
        curr = list(G.neighbors(current_node))
        dest = list(G.neighbors(destination))
        denominator = len(curr)*len(dest)
        if denominator == 0:
            ss_weight = 0
        else:
            numerator = len(set(curr+dest))
            ss_weight = numerator/denominator
    else:
        ss_weight = 0
    return ss_weight 

def preferential_attachment(G, current_node, destination):
    if current_node != destination:
        curr = list(G.neighbors(current_node))
        dest = list(G.neighbors(destination))
        ss_weight = len(curr) * len(dest)
    else:
        ss_weight = 0
    return ss_weight 

def hub_promoted(G, current_node, destination):
    if current_node != destination:
        curr = list(G.neighbors(current_node))
        dest = list(G.neighbors(destination))
        denominator = min(len(curr),len(dest))
        if denominator == 0:
            ss_weight = 0
        else:
            numerator = len(set(curr+dest))
            ss_weight = numerator/denominator
    else:
        ss_weight = 0
    return ss_weight 

def hub_depressed(G, current_node, destination):
    if current_node != destination:
        curr = list(G.neighbors(current_node))
        dest = list(G.neighbors(destination))
        denominator = max(len(curr),len(dest))
        if denominator == 0:
            ss_weight = 0
        else:
            numerator = len(set(curr+dest))
            ss_weight = numerator/denominator
    else:
        ss_weight = 0
    return ss_weight 

def salton_index(G, current_node, destination):
    if current_node != destination:
        curr = list(G.neighbors(current_node))
        dest = list(G.neighbors(destination))
        denominator = math.sqrt(len(curr)*len(dest))
        if denominator == 0:
            ss_weight = 0
        else:
            numerator = len(set(curr+dest))
            ss_weight = numerator/denominator
    else:
        ss_weight = 0
    return ss_weight 

def sorenson_index(G, current_node, destination):
    if current_node != destination:
        curr = list(G.neighbors(current_node))
        dest = list(G.neighbors(destination))
        denominator = len(curr)+len(dest)
        if denominator == 0:
            ss_weight = 0
        else:
            numerator = 2*len(set(curr+dest))
            ss_weight = numerator/denominator
    else:
        ss_weight = 0
    return ss_weight 

def tversky(G, current_node, destination):
    if current_node != destination:
        curr = list(G.neighbors(current_node))
        dest = list(G.neighbors(destination))
        a = set(curr)
        b = set(dest)
        union = a.union(b)
        a_compl = a.difference(b)
        b_compl = b.difference(a)
        denominator = len(union) + len(curr)*len(a_compl)+len(dest)*len(b_compl)
        if denominator == 0:
            ss_weight = 0
        else:
            numerator = len(a.intersection(b))
            ss_weight = numerator/denominator
    else:
        ss_weight = 0
    return ss_weight

def adamic_adar(G, current_node, destination):
    def calc_weight(node):
        denominator = math.log10(len(list(G.neighbors(node))))
        if denominator == 0:
            return 0
        else:
            return 1 / denominator
        
    if current_node != destination:
        curr = list(G.neighbors(current_node))
        dest = list(G.neighbors(destination))
        intersection = set(curr+dest)
        adamic_adar_indices = sum([calc_weight(node) for node in intersection])
        ss_weight = adamic_adar_indices
    else:
        ss_weight = 0
    return ss_weight

def resource_allocation(G, current_node, destination, num_jobs=-1):
    def calc_weight(node):
        denominator = len(list(G.neighbors(node)))
        if denominator == 0:
            return 0
        else:
            return 1 / denominator
        
    if current_node != destination:
        curr = list(G.neighbors(current_node))
        dest = list(G.neighbors(destination))
        intersection = set(curr+dest)
        resource_allocation_indices = sum([calc_weight(node) for node in intersection])
        ss_weight = resource_allocation_indices
    else:
        ss_weight = 0
    return ss_weight

def generate_walks_for_node(node, num_walks, walk_length, probs):
    node_walks = []
    for walk in range(num_walks):
        walk_list = [node]
        for step in range(walk_length - 1):
            neighbors = list(probs[walk_list[-1]].keys())
            probabilities = list(probs[walk_list[-1]].values())
            next_node = np.random.choice(neighbors, p=probabilities)
            walk_list.append(next_node)
        node_walks.append(walk_list)
    return node_walks

In [ ]:
#| export
import networkx as nx
import gensim
import numpy as np
import multiprocessing as mp
import sys
sys.path.append("..")
from wirewalk.core import generate_walks_for_node

class WireWalk():
    def __init__(self, graph: nx.Graph, dimensions: int = 128, window: int = 10, 
                walk_length: int = 80, num_walks: int = 10, 
                workers: int = 1):
        self.graph = graph
        self.dimensions = dimensions
        self.window = window
        self.walk_length = walk_length
        self.num_walks = num_walks
        self.workers = workers
    
    def generate_transition(self, transition_matrix_function):
        G = self.graph
        nodes = G.nodes
        pool = mp.Pool()

        params = []
        for current_node in nodes:
            for destination in nodes:
                params.append((G,current_node, destination))

        ss_weights = pool.starmap(transition_matrix_function, params)
        # close the process pool
        pool.close()
        # wait for all tasks to complete
        pool.join()

        ss_weights_matrix1 = np.reshape(ss_weights, (len(nodes), len(nodes)))
        row_sums = ss_weights_matrix1.sum(axis=1)
        zero_rows = np.where(row_sums == 0)[0]
        ss_weights_matrix1[zero_rows, :] = 0
        for i in zero_rows:
            ss_weights_matrix1[i, i] = 1
        row_sums = ss_weights_matrix1.sum(axis=1)
        ss_weights_matrix1 = np.divide(ss_weights_matrix1, row_sums[:, np.newaxis])
        return ss_weights_matrix1

    def generate_walks(self, transition_probs):
        graph = self.graph
        num_walks = self.num_walks
        walk_length = self.walk_length

        walks = []
        nodes = list(graph.nodes())
        probs = {}
        for i, node_i in enumerate(nodes):
            probs[node_i] = {}
            for j, node_j in enumerate(nodes):
                probs[node_i][node_j] = transition_probs[i][j]

        params = []
        for node in nodes:
            params.append((node, num_walks, walk_length, probs))

        pool = mp.Pool()
        node_walks_list = pool.starmap(generate_walks_for_node, params)
        # close the process pool
        pool.close()
        # wait for all tasks to complete
        pool.join()

        for node_walks in node_walks_list:
            walks += node_walks
            
        return walks

    def fit(self, transformation: callable) -> gensim.models.Word2Vec:
        transition_probability_matrix = self.generate_transition(transformation)
        walks = self.generate_walks(transition_probability_matrix)
        return gensim.models.Word2Vec(walks,window=self.window, workers=self.workers, vector_size=self.dimensions)